CNF Parser

In [1]:
def parse_cnf(in_data):
    cnf = list()
    cnf.append(list())
    maxvar = 0

    for line in in_data:
        tokens = line.split()
        if len(tokens) != 0 and tokens[0] not in ("p", "c", "%"):
            for tok in tokens:
                lit = int(tok)
                maxvar = max(maxvar, abs(lit))
                if lit == 0:
                    cnf.append(list())
                else:
                    lit = lit - 1 if lit > 0 else lit
                    cnf[-1].append(lit)

    assert len(cnf[-1]) == 0
    cnf.pop()
    cnf.pop()
    return cnf
    

In [ ]:
from os import walk
import torch
import random 
import numpy as np
import time 
import torch.nn.functional as F

problems = []
for (dirpath, dirnames, filenames) in walk('sat_problems/uf20-91/'):
# for (dirpath, dirnames, filenames) in walk('sat_problems/uf250-1065/'):
    for f in filenames:
        with open(dirpath + '/' + f, 'r') as file:
            lines = file.readlines()
            
            problems.append(parse_cnf(lines))
            
def create_vars(cnf): 
    maxvar = 0
    for c in cnf:
        for l in c:
            maxvar = max(maxvar, l + 1)
    v_t = torch.tensor(np.random.normal(size=(maxvar,), scale=2.0), requires_grad=True)
    return v_t

# This is where the magic happens
def to_fuzzy(cnf, variables, N, T, S):
    # Create a single vector that first contains the truth values of the variables and then the negated
    # truth values, reversed (so that the indices match: -1 should be at the end in Python)
    variables = torch.cat([variables, reversed(1 - variables)])
    
    # Take, for each conjunct, the correct three truth values. This returns a #clauses x 3 matrix of truth values,
    # where element i, j is the truth values the t the j'th literal of the i'th clause
    indexed = torch.take(variables, cnf)
    
    # Compute the truth value of each disjunct by applying the S norm over the 1st dimension
    conjuncts = S(indexed)
    # Compute the truth value of SAT by applying the T norm over the truth value of the disjuncts
    truth = T(conjuncts)
    return truth

def n_clauses_satisfied(truth_tensor, problem):
    truth = truth_tensor.detach().numpy()
    n = 0
    not_satisfied = []
    for clause in problem:
        sat = False
        for l in clause:
            if l < 0:
                if truth[-l - 1] < 0.5:
                    sat = True
            elif truth[l] >= 0.5:
                sat = True
        if sat:
            n += 1
        else:
            not_satisfied.append(clause)
    return n, not_satisfied

N = lambda a: 1.0 - a
    
def extend_s(s):
    def to_return(a):
        s1 = s(a[:, 0], a[:, 1])
        for i in range(a.size()[1] - 2):
            s1 = s(s1, a[:, i+2])
        return s1
    return to_return

def extend_t(t):
    def to_return(a):
        t1 = t(a[0], a[1])
        for i in range(a.size()[0] - 2):
            t1 = t(t1, a[i+2])
        return t1
    return to_return
    
EPSILON = 0.00000001
# Godel norm
# T = lambda a: torch.min(a)
# S = lambda a: torch.max(a, dim=1)[0]

# Product norm
# T = lambda a: torch.sum(torch.log(a+EPSILON))
# S = lambda a: 1 - torch.prod(1 - a, dim=1)

# Luk
# T = lambda a: torch.clamp(torch.sum(a) + 1 - a.size()[0], min=0)
# S = lambda a: torch.clamp(torch.sum(a, dim=1), max=1)

# nilpotent
def nilpotent_t_norm(a):
    lowest_values = -torch.topk(-a, 2)[0]
    I = 1.0 * (torch.sum(lowest_values) > 1).double()
    return lowest_values[0] * I

def nilpotent_s_norm(a):
    highest_values = torch.topk(a, 2, dim=1)[0]
    I = (torch.sum(highest_values, dim=1) < 1.0).double()
    return highest_values[:, 0] * I + 1 - I

# T = nilpotent_t_norm
# S = nilpotent_s_norm

# Trigonometric
# T = lambda a: 2.0 / np.pi * torch.asin(torch.prod(torch.sin(a * np.pi / 2.0)))
# S = lambda a: 2.0 / np.pi * torch.acos(torch.prod(torch.cos(a * np.pi / 2.0), dim=1))

# Hamacher 0
# T = lambda a: 1 / (1 - a.size()[0] + torch.sum(1 / (a)))
# T = extend_t(lambda a, b: a * b / (a + b - a * b))
# T = lambda a: 1 / (1 - )
# S = extend_s(lambda a, b: (a + b - 2 * a * b) / (1 - a * b + EPSILON))

# Dombi 
# Note: Alpha = 1 gives the Hamacher product (ie, hamacher @ 0)
alpha = 1
# T = lambda a: 1 / (1 + torch.sum(((1-a) / a)**alpha)**(1/alpha))

# Dubois-Prade
# T = lambda a: torch.prod(a) / torch.clamp(torch.max(a), min=0.5)
# S = lambda a: (1 - torch.prod(1 - a, dim=1) - torch.clamp(torch.min(a, dim=1)[0], min=0.5)) / \
#     torch.clamp(torch.max(1 - a, dim=1)[0], min=0.5)

pa = 1
pb0 = -0.5

# pa = 6
# pb0 = -0.1

# Max Sigmoid
# y1 = torch.exp(-pa*(1+pb0)) + 1
# y2 = torch.exp(-pa * (1 - n + pb0)) + 1

# h = y1 / (y2 - y1)
# c = h * y2
# T = lambda a: F.logsigmoid(pa * (torch.sum(a) - a.size()[0] + 1 + pb0))
# S = lambda a: F.sigmoid(pa * (torch.sum(a, dim=1) + pb0))
# T = lambda a: c * F.sigmoid(pa * (torch.sum(a, dim=1) - a.size()[0] + 1 + pb0)) - h

# Prod sigmoid WITHOUT NORMALIZATION!!!!
# T = lambda a: F.logsigmoid(pa * (torch.prod(a) + pb0))
# S = lambda a: F.sigmoid(pa * (1 - torch.prod(1-a, dim=1) + pb0))

# y1 = 1 + torch.exp(-pa*)

# T = lambda a: torch.sigmoid(torch.sum(torch.log(a/(1-a))))
# S = lambda a: torch.sigmoid(torch.sum(torch.log(a/(1-a)), dim=1))

# General sigmoid
def sigmoid(f, pa):
    def _sigmoid(a):
        f0 = f(torch.zeros_like(a))
        f1 = f(torch.ones_like(a))
        y1 = torch.exp(-pa * (f1 + pb0)) + 1
        y2 = torch.exp(-pa * (f0 + pb0)) + 1
        return (y1/(y2 - y1)) * (y2 * torch.sigmoid(pa*(f(a) + pb0)) - 1)
    return _sigmoid

# T = sigmoid(lambda a: torch.sum(a) - a.size()[0] + 1)
S = sigmoid(lambda a: 1 - torch.prod(1-a, dim=1), pa)

# Yager
p = 2
# T = lambda a: torch.clamp(1-torch.sum((1-a)**p)**(1/p), min=0)
# S = lambda a: torch.clamp(torch.sum(a**p, dim=1)**(1/p), max=1)
# T = sigmoid(T, pa)

# RMSE
T = lambda a: 1-(EPSILON + torch.mean((1-a)**p))**(1/p)  
# S = lambda a: torch.mean(a**p, dim=1)**(1/p)

# MAE (Mean absolute error)
# T = lambda a: 1 - torch.mean(1 - a)
# T = lambda a: torch.mean(a)

clauses_satisfied = []
n_solved = 0
solved_in = []

for problem_n, problem in enumerate(problems):
    print(problem_n, '------------------------')
    start = time.time()
    logit_variables = create_vars(problem)
    optimizer = torch.optim.SGD([logit_variables], lr=10, momentum=0.9)
    problem = torch.tensor(problem)
    for iteration in range(3000):
        truth_variables = torch.sigmoid(logit_variables)
#         truth_variables = torch.clamp(logit_variables, min=0, max=1)
        if iteration % 100 == 0:
            n_c_sat, unsatisfied = n_clauses_satisfied(truth_variables, problem)
            if n_c_sat == len(problem):
                print('Solution found in', iteration, 'iterations')
                n_solved += 1
                solved_in.append(iteration)
                break
        optimizer.zero_grad()
        truth = -to_fuzzy(problem, truth_variables, N, T, S)
        if iteration % 1000 == 0:
            print('iteration', iteration)
            print('truth', truth.detach().numpy())
            print('n satisfied', n_c_sat)
        truth.backward()
        optimizer.step()
#     print('Solution:', torch.sigmoid(logit_variables))
    print('Time taken:', time.time() - start)
    print('Clauses satisfied:', n_c_sat)
    clauses_satisfied.append(n_c_sat)
print('Solved:', n_solved)
print('Avg solved clauses', np.mean(clauses_satisfied))
print('Avg iterations to solve', np.mean(solved_in))

0 ------------------------
iteration 0
truth -0.8013504280286926
n satisfied 78
iteration 1000
truth -0.9016298039186504
n satisfied 89
iteration 2000
truth -0.9016321249376158
n satisfied 89
Time taken: 1.8781077861785889
Clauses satisfied: 89
1 ------------------------
iteration 0
truth -0.8370198387477342
n satisfied 81
Solution found in 100 iterations
Time taken: 0.05974292755126953
Clauses satisfied: 91
2 ------------------------
iteration 0
truth -0.8333511868976421
n satisfied 87
iteration 1000
truth -0.9316553184255187
n satisfied 90
iteration 2000
truth -0.9316674529931548
n satisfied 90
Time taken: 1.8498256206512451
Clauses satisfied: 90
3 ------------------------
iteration 0
truth -0.8000812756253863
n satisfied 83
iteration 1000
truth -0.9000364536228094
n satisfied 88
iteration 2000
truth -0.9000377754862435
n satisfied 88
Time taken: 1.8510615825653076
Clauses satisfied: 88
4 ------------------------
iteration 0
truth -0.7426579069364663
n satisfied 74
Solution found in 

iteration 1000
truth -0.9297567657632746
n satisfied 90
iteration 2000
truth -0.9297642158773275
n satisfied 90
Time taken: 2.5377209186553955
Clauses satisfied: 90
42 ------------------------
iteration 0
truth -0.799835837287883
n satisfied 81
Solution found in 100 iterations
Time taken: 0.07189083099365234
Clauses satisfied: 91
43 ------------------------
iteration 0
truth -0.7764226641930578
n satisfied 79
iteration 1000
truth -0.9122147956969272
n satisfied 89
iteration 2000
truth -0.9122163916865383
n satisfied 89
Time taken: 2.0048935413360596
Clauses satisfied: 89
44 ------------------------
iteration 0
truth -0.8279395738834461
n satisfied 86
Solution found in 100 iterations
Time taken: 0.09428906440734863
Clauses satisfied: 91
45 ------------------------
iteration 0
truth -0.8017665560705336
n satisfied 80
iteration 1000
truth -0.9311251770514922
n satisfied 89
iteration 2000
truth -0.9311455752546455
n satisfied 89
Time taken: 3.2537131309509277
Clauses satisfied: 89
46 -----

Time taken: 2.468212366104126
Clauses satisfied: 89
83 ------------------------
iteration 0
truth -0.8048194346508301
n satisfied 79
Solution found in 200 iterations
Time taken: 0.12766790390014648
Clauses satisfied: 91
84 ------------------------
iteration 0
truth -0.8361086083080083
n satisfied 85
iteration 1000
truth -0.926350507424109
n satisfied 90
iteration 2000
truth -0.9263564550790855
n satisfied 90
Time taken: 1.6379351615905762
Clauses satisfied: 90
85 ------------------------
iteration 0
truth -0.7749831172565942
n satisfied 77
Solution found in 100 iterations
Time taken: 0.12193822860717773
Clauses satisfied: 91
86 ------------------------
iteration 0
truth -0.823853386499942
n satisfied 83
Solution found in 100 iterations
Time taken: 0.05701184272766113
Clauses satisfied: 91
87 ------------------------
iteration 0
truth -0.7795480966932544
n satisfied 79
Solution found in 100 iterations
Time taken: 0.06706428527832031
Clauses satisfied: 91
88 ------------------------
iter

Time taken: 1.3465230464935303
Clauses satisfied: 89
125 ------------------------
iteration 0
truth -0.7631509559264164
n satisfied 78
Solution found in 100 iterations
Time taken: 0.05180692672729492
Clauses satisfied: 91
126 ------------------------
iteration 0
truth -0.8223874324230895
n satisfied 78
iteration 1000
truth -0.936043487472462
n satisfied 90
iteration 2000
truth -0.9360572538869154
n satisfied 90
Time taken: 1.8673667907714844
Clauses satisfied: 90
127 ------------------------
iteration 0
truth -0.7935728180159649
n satisfied 78
iteration 1000
truth -0.9398444517343425
n satisfied 90
iteration 2000
truth -0.9398536655716246
n satisfied 90
Time taken: 1.7862999439239502
Clauses satisfied: 90
128 ------------------------
iteration 0
truth -0.7964081980083708
n satisfied 82
iteration 1000
truth -0.9105001202502918
n satisfied 89
iteration 2000
truth -0.9105036760941387
n satisfied 89
Time taken: 1.4960308074951172
Clauses satisfied: 89
129 ------------------------
iteration

iteration 1000
truth -0.9137708421475897
n satisfied 89
iteration 2000
truth -0.9137737653900408
n satisfied 89
Time taken: 1.638612985610962
Clauses satisfied: 89
164 ------------------------
iteration 0
truth -0.734051666713962
n satisfied 78
iteration 1000
truth -0.9314472004902693
n satisfied 90
iteration 2000
truth -0.9314537183090952
n satisfied 90
Time taken: 1.864732027053833
Clauses satisfied: 90
165 ------------------------
iteration 0
truth -0.7708172534495286
n satisfied 77
Solution found in 100 iterations
Time taken: 0.06420230865478516
Clauses satisfied: 91
166 ------------------------
iteration 0
truth -0.784517901098074
n satisfied 78
Solution found in 200 iterations
Time taken: 0.0942232608795166
Clauses satisfied: 91
167 ------------------------
iteration 0
truth -0.812478241476355
n satisfied 83
iteration 1000
truth -0.9366523194884834
n satisfied 90
iteration 2000
truth -0.9366610534197705
n satisfied 90
Time taken: 1.7264628410339355
Clauses satisfied: 90
168 -----

...

iteration 2000
truth -0.9133904584149374
n satisfied 89
Time taken: 1.43394136428833
Clauses satisfied: 89
572 ------------------------
iteration 0
truth -0.8138251534981311
n satisfied 79
Solution found in 100 iterations
Time taken: 0.046556711196899414
Clauses satisfied: 91
573 ------------------------
iteration 0
truth -0.8481454946463856
n satisfied 85
iteration 1000
truth -0.9372068799455185
n satisfied 90
iteration 2000
truth -0.93721384282493
n satisfied 90
Time taken: 1.3620529174804688
Clauses satisfied: 90
574 ------------------------
iteration 0
truth -0.7487896785567573
n satisfied 75
Solution found in 100 iterations
Time taken: 0.04232072830200195
Clauses satisfied: 91
575 ------------------------
iteration 0
truth -0.7541607628488398
n satisfied 77
iteration 1000
truth -0.9351797606447094
n satisfied 90
iteration 2000
truth -0.935234728355417
n satisfied 90
Time taken: 1.9909358024597168
Clauses satisfied: 90
576 ------------------------
iteration 0
truth -0.7793076914613

Time taken: 1.4697706699371338
Clauses satisfied: 90
614 ------------------------
iteration 0
truth -0.7444325778842291
n satisfied 78
iteration 1000
truth -0.936363556657632
n satisfied 90
iteration 2000
truth -0.9363695899695621
n satisfied 90
Time taken: 1.4820566177368164
Clauses satisfied: 90
615 ------------------------
iteration 0
truth -0.7846863659369784
n satisfied 75
Solution found in 100 iterations
Time taken: 0.05279946327209473
Clauses satisfied: 91
616 ------------------------
iteration 0
truth -0.8005936754870531
n satisfied 80
iteration 1000
truth -0.9315593817993953
n satisfied 90
iteration 2000
truth -0.9315651502091797
n satisfied 90
Time taken: 1.4604241847991943
Clauses satisfied: 90
617 ------------------------
iteration 0
truth -0.7829599002554632
n satisfied 84
iteration 1000
truth -0.9111586126877159
n satisfied 89
iteration 2000
truth -0.9111793352398333
n satisfied 89
Time taken: 1.4482598304748535
Clauses satisfied: 89
618 ------------------------
iteration

iteration 2000
truth -0.926154493367425
n satisfied 90
Time taken: 1.529303789138794
Clauses satisfied: 90
653 ------------------------
iteration 0
truth -0.7789742252638596
n satisfied 80
iteration 1000
truth -0.9096692119219685
n satisfied 89
iteration 2000
truth -0.9096709303606227
n satisfied 89
Time taken: 1.8554317951202393
Clauses satisfied: 89
654 ------------------------
iteration 0
truth -0.790585525884283
n satisfied 76
Solution found in 100 iterations
Time taken: 0.04249143600463867
Clauses satisfied: 91
655 ------------------------
iteration 0
truth -0.8308172161356175
n satisfied 85
iteration 1000
truth -0.9385791109406402
n satisfied 90
iteration 2000
truth -0.9385906252997058
n satisfied 90
Time taken: 1.2204115390777588
Clauses satisfied: 90
656 ------------------------
iteration 0
truth -0.7458660888287014
n satisfied 75
iteration 1000
truth -0.9337440558142419
n satisfied 90
iteration 2000
truth -0.9337905999870302
n satisfied 90
Time taken: 1.4075994491577148
Clause

iteration 2000
truth -0.9370334172000662
n satisfied 90
Time taken: 2.624082326889038
Clauses satisfied: 90
693 ------------------------
iteration 0
truth -0.7523534264009429
n satisfied 75
Solution found in 100 iterations
Time taken: 0.18628883361816406
Clauses satisfied: 91
694 ------------------------
iteration 0
truth -0.7714617209059593
n satisfied 80
Solution found in 100 iterations
Time taken: 0.06176304817199707
Clauses satisfied: 91
695 ------------------------
iteration 0
truth -0.8230383612970344
n satisfied 84
Solution found in 100 iterations
Time taken: 0.054468393325805664
Clauses satisfied: 91
696 ------------------------
iteration 0
truth -0.7880063128476053
n satisfied 75
Solution found in 100 iterations
Time taken: 0.06869292259216309
Clauses satisfied: 91
697 ------------------------
iteration 0
truth -0.8017727384861647
n satisfied 80
Solution found in 100 iterations
Time taken: 0.1465597152709961
Clauses satisfied: 91
698 ------------------------
iteration 0
truth 

Solution found in 400 iterations
Time taken: 0.1827995777130127
Clauses satisfied: 91
734 ------------------------
iteration 0
truth -0.783305770367699
n satisfied 79
Solution found in 100 iterations
Time taken: 0.0439300537109375
Clauses satisfied: 91
735 ------------------------
iteration 0
truth -0.8228235562713753
n satisfied 76
iteration 1000
truth -0.9399644368748721
n satisfied 90
iteration 2000
truth -0.9400569501781623
n satisfied 90
Time taken: 1.504765510559082
Clauses satisfied: 90
736 ------------------------
iteration 0
truth -0.7532962864004766
n satisfied 79
iteration 1000
truth -0.9252092431666401
n satisfied 90
iteration 2000
truth -0.9253078872173643
n satisfied 90
Time taken: 1.7625908851623535
Clauses satisfied: 90
737 ------------------------
iteration 0
truth -0.7584446286913393
n satisfied 77
Solution found in 300 iterations
Time taken: 0.45316314697265625
Clauses satisfied: 91
738 ------------------------
iteration 0
truth -0.7926543344491467
n satisfied 80
Sol

In [3]:
def sigmoid(f):
    def _sigmoid(a, b):
        return f(a, b)
    return _sigmoid

S_SP = sigmoid(lambda a, b: 1 - (1-a) * (1 - b))
I_SP = sigmoid(lambda a, b: 1 - a + a * b)

print(S_SP(1, 0))
print(I_SP(1, 0))

1
0
